kdv eqn:
$$u_t +uu_x +u_{xxx} = 0$$

where u_x = first derivative of u wrt x

## Variables  

### u_only_mat_reshaped (size: 8 x 10 x 400) 
- only contains the u variable
- First dimension is the number of cases (8)
- The second dimension is the number of time snapshots (10)
- The third dimension corresponds to the number of space points (400)

### u_mat (size: 32000 × 5)  
- Each row corresponds to a point in space and time for a given case.  
- Each column contains one of the following values:  
  1. u — solution value  
  2. ux — first spatial derivative (∂u/∂x)  
  3. uxx — second spatial derivative (∂²u/∂x²)  
  4. uxxx — third spatial derivative (∂³u/∂x³)  
  5. uxxxx — fourth spatial derivative (∂⁴u/∂x⁴)  

#### Structure
- Data is organised sequentially over space, time, and cases.  
- Dimensions:  
  - Space (N): 400 points  
  - Time (Nt): 10 snapshots  
  - Cases (N_case): 8 independent cases  
- Overall shape: 400 × 10 × 8 = 32000 rows  

Row ordering follows:  

Case 1
u(x0,t0)   ux(x0,t0)   uxx(x0,t0)   uxxx(x0,t0)   uxxxx(x0,t0)
u(x1,t0)   ux(x1,t0)   uxx(x1,t0)   uxxx(x1,t0)   uxxxx(x1,t0)
...
u(x399,t0) ...
u(x0,t1)   ...
...
u(x399,t9) ...
Case 2
...
Case 8
...

---

### u_tar (size: 32000 × 1)  
- Contains the time derivative:  
  - ut — ∂u/∂t  
- Follows the same row ordering as u_mat.  

---

✅ In short:  
- u_mat provides the state (u and spatial derivatives).  
- u_tar provides the target (time derivative).  
- Together, they represent spatio-temporal samples from 8 cases of the KdV equation.  

---

## Tensor Reshaping Illustration

Think of the data as a 3D tensor:

    [Space = 400] × [Time = 10] × [Cases = 8] = 32000 samples

Each row in u_mat and u_tar corresponds to one slice of this tensor, flattened into a 2D matrix.


# first task , have u, u_t, u_x,u_xx,...

In [3]:
import pysr
import sympy
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from pysr import PySRRegressor, TemplateExpressionSpec, TensorBoardLoggerSpec
from sklearn.model_selection import train_test_split
from scipy.io import loadmat


# Set up default parameters we'll reuse
default_pysr_params = dict(
    populations=30,
    model_selection="best",
)

In [4]:
KdV_data = loadmat('./kdv_data_for_workshop.mat')


In [5]:
KdV_data.keys()

dict_keys(['__header__', '__version__', '__globals__', 'N', 'N_case', 'Nt', 't_vec', 'u_mat', 'u_tar', 'x_vec'])

In [13]:
target = KdV_data['u_tar'].flatten()
data = KdV_data['u_mat']

In [16]:
target.shape, data.shape

((32000,), (32000, 5))

In [11]:
print(KdV_data['u_mat'].shape)
print(KdV_data['u_tar'].shape)

(32000, 5)
(32000, 1)


In [23]:
var_names = ['u', 'u_x', 'u_xx', 'u_xxx', 'u_xxxx']

In [24]:
model = PySRRegressor(
    binary_operators=["+", "-", "*", "/"],
    unary_operators=["exp", "log", "sin", "cos", "square", "cube", "sqrt"],
    # nested_constraints={"sin": {"sin": 0, "cos": 0}, "cos": {"sin": 0, "cos": 0}},
    niterations=150,
    maxsize=22,
    variable_names=var_names,
    batching = True,
    batch_size=16,
    # turbo=True,
    populations=24,
    model_selection="best",
)

/Users/ash/Library/CloudStorage/OneDrive-Nexus365/PHD/Symbolic_discovery/.venv/lib/python3.12/site-packages/pysr/sr.py:1046: FutureWarning: `variable_names` is a data-dependent parameter and should be passed when fit is called. Ignoring parameter; please pass `variable_names` during the call to fit instead.
  warnings.warn(


In [26]:
model.fit(data, target, variable_names=var_names)

/Users/ash/Library/CloudStorage/OneDrive-Nexus365/PHD/Symbolic_discovery/.venv/lib/python3.12/site-packages/pysr/sr.py:2217: UserWarning: The discovered expressions are being reset. Please set `warm_start=True` if you wish to continue to start a search where you left off.
  warnings.warn(
/Users/ash/Library/CloudStorage/OneDrive-Nexus365/PHD/Symbolic_discovery/.venv/lib/python3.12/site-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 9.370e+03
Progress: 1179 / 3600 total iterations (32.750%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           2.328e+10  0.000e+00  y = -2.1142
2           2.328e+10  -0.000e+00  y = log(u_xxxx)
3           7.126e+08  -0.000e+00  y = u_x * -266.75
7           1.596e-04  6.235e+00  y = ((-3.0622e-06 - u) * u_x) - u_xxx
9           1.596e-04  8.941e-08  y = ((-3.1203e-06 - u) * u_x) - (u_xxx + -1.3613e-06)
11          1.596e-04  2.539e-05  y = ((-2.6315e-06 - u) * u_x) + (1.1784 - (u_xxx + 1.1785)...
                                      )
───────────────────────────────────────────────────────────────────────────────────────────────────
════════════════════════════════════════════════════════════════════════════════════════════════════
Press 'q' and t

[ Info: Final population:
[ Info: Results saved to:


,model_selection,'best'
,binary_operators,"['+', '-', ...]"
,unary_operators,"['exp', 'log', ...]"
,expression_spec,None
,niterations,150
,populations,24
,population_size,27
,max_evals,None
,maxsize,22
,maxdepth,None
,warmup_maxsize_by,None


  - outputs/20250922_142900_a6a5Cs/hall_of_fame.csv


In [27]:
model.equations_

,complexity,loss,equation,score,sympy_format,lambda_format
0,1,2.328078e+10,-2.1141942,0.000000e+00,-2.11419420000000,PySRFunction(X=>-2.11419420000000)
1,2,2.328076e+10,log(u_xxxx),9.879398e-07,log(u_xxxx),PySRFunction(X=>log(u_xxxx))
2,3,7.118991e+08,u_x * -268.21207,3.487446e+00,u_x*(-268.21207),PySRFunction(X=>u_x*(-268.21207))
3,6,7.115906e+08,u_x * (cos(u_x) + -267.9196),1.444806e-04,u_x*(cos(u_x) - 267.9196),PySRFunction(X=>u_x*(cos(u_x) - 267.9196))
4,7,1.595570e-04,((-2.3641226e-6 - u) * u_x) - u_xxx,2.912612e+01,u_x*(-u - 2.3641226e-6) - u_xxx,PySRFunction(X=>u_x*(-u - 2.3641226e-6) - u_xxx)
5,9,1.594715e-04,((u - u_xxx) - u) - (u_x * u),2.678757e-04,-u*u_x - u + u - u_xxx,PySRFunction(X=>-u*u_x - u + u - u_xxx)
6,11,1.594715e-04,((-6.2258914e-7 - u) * u_x) - (u + (u_xxx - u)),9.406068e-08,u_x*(-u - 6.2258914e-7) - (-u + u + u_xxx),PySRFunction(X=>u_x*(-u - 6.2258914e-7) - (-u ...
7,13,1.585618e-04,(((u_x - u_x) - u_x) * u) - (u_xxx + (u_x * -2...,2.860368e-03,u*(-u_x - u_x + u_x) - (u_x*(-2.388702e-6) + u...,PySRFunction(X=>u*(-u_x - u_x + u_x) - (u_x*(-...
8,15,1.585053e-04,((u_x - u_x) - (u * u_x)) - (u_xxx + ((2.43752...,1.781326e-04,-u*u_x - u_x + u_x - (u_xxx + (2.4375262 - u_x...,PySRFunction(X=>-u*u_x - u_x + u_x - (u_xxx + ...
9,16,1.584875e-04,((cube(0.74462295 - 0.75664324) - u) * u_x) - ...,1.126839e-04,u_x*(-u + (0.74462295 - 1*0.75664324)**3) - (u...,PySRFunction(X=>u_x*(-u + (0.74462295 - 1*0.75...


In [28]:
model.sympy()

u_x*(-u - 2.3641226e-6) - u_xxx

# second task, dont have u_x,u_xx,... need to calculate myself but will then have errors/noise from calculating the spatial derivatives

In [ ]:
data = KdV_data['u_mat'][:,0]
u_reshape = data.reshape(8,10,400)
target = KdV_data['u_tar']
var_names = ['u']
u_reshape.shape, target.shape
#u_shape = cases x time x space

((8, 10, 400), (32000, 1))

In [52]:
# Extract spatial coordinates
x = KdV_data['x_vec'].flatten()


In [54]:

# Calculate spatial and time derivatives
def calculate_derivatives(u, x, t):
    """
    Calculate spatial and time derivatives for u data
    u: numpy array of shape (cases, time, space)
    x: spatial coordinates
    t: time coordinates
    
    Returns:
    u_t: time derivative (cases, time-2, space)
    u_x: first spatial derivative (cases, time, space-2)
    u_xx: second spatial derivative (cases, time, space-4)
    u_xxx: third spatial derivative (cases, time, space-6)
    """
    n_cases, n_time, n_space = u.shape
    
    # Calculate time derivative using central difference
    dt = t[1] - t[0]
    u_t = (u[:, 2:, :] - u[:, :-2, :]) / (2 * dt)
    
    # Calculate spatial derivatives using central difference
    dx = x[1] - x[0]
    
    # First derivative: u_x
    u_x = np.zeros((n_cases, n_time, n_space-2))
    u_x = (u[:, :, 2:] - u[:, :, :-2]) / (2 * dx)
    
    # Second derivative: u_xx
    u_xx = np.zeros((n_cases, n_time, n_space-4))
    u_xx = (u[:, :, 4:] - 2*u[:, :, 2:-2] + u[:, :, :-4]) / (dx**2)
    
    # Third derivative: u_xxx
    u_xxx = np.zeros((n_cases, n_time, n_space-6))
    u_xxx = (u[:, :, 6:] - 3*u[:, :, 4:-2] + 3*u[:, :, 2:-4] - u[:, :, :-6]) / (dx**3)
    
    return u_t, u_x, u_xx, u_xxx

# Calculate derivatives
u_t, u_x, u_xx, u_xxx = calculate_derivatives(u_reshape, x, t)

# Print shapes to verify
print(f"Original u shape: {u_reshape.shape}")
print(f"u_t shape: {u_t.shape}")
print(f"u_x shape: {u_x.shape}")
print(f"u_xx shape: {u_xx.shape}")
print(f"u_xxx shape: {u_xxx.shape}")

Original u shape: (8, 10, 400)
u_t shape: (8, 8, 400)
u_x shape: (8, 10, 398)
u_xx shape: (8, 10, 396)
u_xxx shape: (8, 10, 394)


In [51]:
print(u_reshape)

[[[40.85690899 43.12149645 45.49592102 ... 34.68479689 36.64228895
   38.69841905]
  [38.34657235 40.35754739 42.45566435 ... 32.81412393 34.57715169
   36.42052945]
  [36.23460705 38.04761743 39.93201804 ... 31.21015497 32.81732144
   34.49164568]
  ...
  [29.04775436 30.32030542 31.62479676 ... 25.59664616 26.75245564
   27.88280089]
  [28.10803865 29.26765812 30.39252717 ... 24.7890207  25.79470223
   26.9157246 ]
  [27.1444298  28.20493062 29.36738455 ... 23.97389284 25.03140434
   26.12365302]]

 [[44.77777402 47.05556561 49.43302468 ... 38.51341219 40.50962063
   42.59676416]
  [42.01604518 44.03926547 46.14172247 ... 36.40462045 38.20049787
   40.0703721 ]
  [39.69085642 41.51501542 43.40418006 ... 34.59800006 36.23345192
   37.93069256]
  ...
  [31.81598109 33.05714729 34.33214897 ... 28.27951311 29.42808169
   30.60912715]
  [30.70591354 31.87426208 33.06487394 ... 27.36267648 28.4406125
   29.55775602]
  [29.67558283 30.79247666 31.93274716 ... 26.52563567 27.5562456
   28.59

In [ ]:
def time_derivative(u, t):
    dt = t[1] - t[0]
    return (u[:,2:,:] - u[:,:-2,:]) / (2*dt)

def spatial_derivative(u, x):
    dx = x[1] - x[0]
    return (u[:,:,2:] - u[:,:,:-2]) / (2*dx)

# u_reshape_target = ??

In [ ]:
u_x = spatial_derivative(u_reshape, x)

In [47]:
t = KdV_data['t_vec'].flatten()
t = t[0:10] #just consider first 10 time points
print(t)
t.shape

[0.         0.00031837 0.00063673 0.0009551  0.00127347 0.00159184
 0.0019102  0.00222857 0.00254694 0.00286531]


(10,)

In [ ]:
#calculate the time and spatial derivatives from this.

want to calculate the time and spatial derivatives, reshape into dimensions x, t, case